# First Marlowe Using Plutus V2 on Mainnet

## Record version numbers

In [1]:
git rev-parse HEAD

54f956b5ba3009a41ecae11d502c165021ceb1ad


In [2]:
marlowe-cli --version

marlowe-cli 0.0.8.2


In [3]:
cardano-cli --version

cardano-cli 1.35.3 - linux-x86_64 - ghc-8.10
git rev 0000000000000000000000000000000000000000


## Select `mainnet`

In [4]:
export CARDANO_TESTNET_MAGIC=764824073
export CARDANO_NODE_SOCKET_PATH=node.socket

## Set path to keys

In [5]:
export TREASURY=treasury

In [6]:
PAYMENT_SKEY="$TREASURY/payment.skey"
PAYMENT_ADDR="$(cat $TREASURY/payment.mainnet.address)"
echo "$PAYMENT_ADDR"

addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h


## Check for collateral

In [7]:
cardano-cli query utxo --mainnet --address "$PAYMENT_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
0d3e697efb2e9c5ea7ce5008d9804fc5188a584ead7882c2fc67757d6a88b226     0        48678354 lovelace + TxOutDatumNone


## Find current time

In [8]:
NOW=$(($(date -u +%s) * 1000))
echo "$(date -u) = $NOW"

Sun Sep 25 04:47:09 PM UTC 2022 = 1664124429000


In [9]:
SECOND=1000
MINUTE=$((60 * SECOND))
HOUR=$((60 * MINUTE))
DAY=$((24 * HOUR))

## Check for required input files

In [10]:
json2yaml token-drop-mainnet-1.marlowe | head

era: babbage
plutusVersion: PlutusScriptV2
tx:
  continuations: []
  contract:
    timeout: 1665178603000
    timeout_continuation: close
    when:
    - case:
        notify_if:


In [11]:
json2yaml token-drop-mainnet-2.message

'674':
  msg:
  - Welcome, Marlowe, to the Plutus V2!


## Check that the contract isn't malicious

In [12]:
jq .tx.contract token-drop-mainnet-1.marlowe | json2yaml

timeout: 1665178603000
timeout_continuation: close
when:
- case:
    notify_if:
      ge_than: 1663882603000
      value: time_interval_start
  then:
    from_account:
      address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h
    pay: 1
    then:
      from_account:
        address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h
      pay: 1
      then:
        from_account:
          address: addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h
        pay: 1
        then: close
        to:
          party:
            address: addr1qy4mhhat3s522uplguhkk2xxrnwsv65e0yxk9nyrn5530ksvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sc8hwky
        token:
          currency_symbol: 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d
          token_name: M4B
      to:
        party:
          address: addr1q8se6c0am4wjfpxgwdct92tugjedfuqj77zecttxttj5620vdnv0lax0mhmmau26npwqg47drdwqku00s580409jvwgs3nmzq4
      token:
        currency_symbol: 8bb3b343d8e40447

## Prepare the transaction

In [22]:
NOW=$(($(date -u +%s) * 1000))
echo "$(date -u) = $NOW"

Tue Sep 27 07:22:16 PM UTC 2022 = 1664306536000


In [23]:
SECOND=1000
MINUTE=$((60 * SECOND))
HOUR=$((60 * MINUTE))
DAY=$((24 * HOUR))

In [24]:
marlowe-cli run prepare --marlowe-file token-drop-mainnet-1.marlowe \
                        --notify \
                        --invalid-before    "$NOW" \
                        --invalid-hereafter "$((NOW + 30 * MINUTE))" \
                        --out-file token-drop-mainnet-2.marlowe

Payment 1
  Acccount: "\"addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h\""
  Payee: Party "\"addr1qxfqtdclx4sl6evhsgtjcdz300y9ncl2vvft4nnlw9p72uqqnhdfgfcjm5nket22a7g8pxepf444uvckxxevml48gh9sx8uvy3\""
  Ada: Lovelace {getLovelace = 0}
  8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d."M4B": 1
Payment 2
  Acccount: "\"addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h\""
  Payee: Party "\"addr1q8se6c0am4wjfpxgwdct92tugjedfuqj77zecttxttj5620vdnv0lax0mhmmau26npwqg47drdwqku00s580409jvwgs3nmzq4\""
  Ada: Lovelace {getLovelace = 0}
  8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d."M4B": 1
Payment 3
  Acccount: "\"addr1vy9prvx8ufwutkwxx9cmmuuajaqmjqwujqlp9d8pvg6gupceql82h\""
  Payee: Party "\"addr1qy4mhhat3s522uplguhkk2xxrnwsv65e0yxk9nyrn5530ksvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sc8hwky\""
  Ada: Lovelace {getLovelace = 0}
  8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d."M4B": 1
Payment 4
  Acccount: "\"addr1vy9prvx8ufwutkwxx9cmmuuajaqm

## Verify that the UTxO is at the Marlowe script address

In [14]:
SCRIPT_ADDR=$(jq -r '.tx.marloweValidator.address' token-drop-mainnet-1.marlowe)
echo "$SCRIPT_ADDR"

addr1w94f8ywk4fg672xasahtk4t9k6w3aql943uxz5rt62d4dvq8evxaf


In [15]:
TX_MARLOWE="9c91bdbbface40d0f6f9099a5888635bfe9ba72873e8d21e3bbc541fb53159b4#1"

In [16]:
cardano-cli query utxo --mainnet --address "$SCRIPT_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
9c91bdbbface40d0f6f9099a5888635bfe9ba72873e8d21e3bbc541fb53159b4     1        3000000 lovelace + 3 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d3442 + TxOutDatumHash ScriptDataInBabbageEra "605635647b7fa1f331978bdf5aeed389bdb3369274482ead60c6b86c389cf020"


## Check the script address

In [17]:
marlowe-cli util decode-bech32 "$SCRIPT_ADDR"

Human-readable part: addr
716a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0


In [18]:
marlowe-cli util decode-bech32 "$(marlowe-cli contract address)"

Human-readable part: addr_test
706a9391d6aa51af28dd876ebb5565b69d1e83e5ac7861506bd29b56b0


These differ only by the `mainnet`/`testnet` bit and the value in the `.marlowe` file has the correct network bit (`mainnet = 1`), so we are *probably* okay here.

## Submit the transaction

Loop until the Plutus V2 cost model is on the blockchain.

In [36]:
for x in `seq 1 1800`
do

echo

NOW=$(($(date -u +%s) * 1000))
echo "$(date -u) = $NOW"

marlowe-cli run prepare --marlowe-file token-drop-mainnet-1.marlowe \
                        --notify \
                        --invalid-before    "$NOW" \
                        --invalid-hereafter "$((NOW + 30 * MINUTE))" \
                        --out-file token-drop-mainnet-2.marlowe \
>& /dev/null

marlowe-cli run auto-execute --tx-in-marlowe "$TX_MARLOWE" \
                             --marlowe-in-file token-drop-mainnet-1.marlowe \
                             --marlowe-out-file token-drop-mainnet-2.marlowe \
                             --change-address "$PAYMENT_ADDR" \
                             --required-signer "$PAYMENT_SKEY" \
                             --metadata-file token-drop-mainnet-2.message \
                             --out-file token-drop-mainnet-2.txbody \
                             --submit 1800 \
                             --print-stats

if [ $? -eq 0 ]
then
  echo "Success!"
  break
fi

sleep 2s

done


Tue Sep 27 09:44:54 PM UTC 2022 = 1664315094000
marlowe-cli: (Array.!): undefined array element

Tue Sep 27 09:44:57 PM UTC 2022 = 1664315097000
marlowe-cli: (Array.!): undefined array element

Tue Sep 27 09:45:01 PM UTC 2022 = 1664315101000
marlowe-cli: (Array.!): undefined array element

Tue Sep 27 09:45:05 PM UTC 2022 = 1664315105000
marlowe-cli: (Array.!): undefined array element

Tue Sep 27 09:45:08 PM UTC 2022 = 1664315108000
marlowe-cli: (Array.!): undefined array element

Tue Sep 27 09:45:12 PM UTC 2022 = 1664315112000
marlowe-cli: (Array.!): undefined array element

Tue Sep 27 09:45:16 PM UTC 2022 = 1664315116000
marlowe-cli: (Array.!): undefined array element

Tue Sep 27 09:45:19 PM UTC 2022 = 1664315119000
marlowe-cli: (Array.!): undefined array element

Tue Sep 27 09:45:23 PM UTC 2022 = 1664315123000
marlowe-cli: (Array.!): undefined array element

Tue Sep 27 09:45:27 PM UTC 2022 = 1664315127000
marlowe-cli: (Array.!): undefined array element

Tue Sep 27 09:45:30 PM UTC 20

### Check that there is no UTxO at the script address

In [37]:
cardano-cli query utxo --mainnet --address "$SCRIPT_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------


### Check that the recipients received their tokens

In [38]:
RECIPIENT_1_ADDR=addr1qxfqtdclx4sl6evhsgtjcdz300y9ncl2vvft4nnlw9p72uqqnhdfgfcjm5nket22a7g8pxepf444uvckxxevml48gh9sx8uvy3
RECIPIENT_2_ADDR=addr1q8se6c0am4wjfpxgwdct92tugjedfuqj77zecttxttj5620vdnv0lax0mhmmau26npwqg47drdwqku00s580409jvwgs3nmzq4
RECIPIENT_3_ADDR=addr1qy4mhhat3s522uplguhkk2xxrnwsv65e0yxk9nyrn5530ksvluken35ncjnu0puetf5jvttedkze02d5kf890kquh60sc8hwky

In [39]:
cardano-cli query utxo --mainnet --address "$RECIPIENT_1_ADDR" --address "$RECIPIENT_2_ADDR" --address "$RECIPIENT_3_ADDR"

                           TxHash                                 TxIx        Amount
--------------------------------------------------------------------------------------
53cd9aa80d8f046ce2bdb6c13e8a904cd83aaa6afac07dc2a52f4799b5fa8ed1     2        1142150 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d3442 + TxOutDatumNone
53cd9aa80d8f046ce2bdb6c13e8a904cd83aaa6afac07dc2a52f4799b5fa8ed1     3        1142150 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d3442 + TxOutDatumNone
53cd9aa80d8f046ce2bdb6c13e8a904cd83aaa6afac07dc2a52f4799b5fa8ed1     4        1142150 lovelace + 1 8bb3b343d8e404472337966a722150048c768d0a92a9813596c5338d.4d3442 + TxOutDatumNone


**Voilà!**